This script opens and extracts nino-3 data and stores it as a netcdf file.

In [5]:
# import needed modules
import pandas as pd
import xarray as xr
import os

In [2]:
misc_data_path = '/glade/u/home/zcleveland/scratch/misc_data/'  # path to misc data

In [15]:
# open nino-3 .csv file and convert it to netcdf
def nino_3_extractor(overwrite_flag=False):

    # define in and out paths
    in_fp = f'{misc_data_path}nino-3/nino-3_1980-2019.csv'
    out_fp = f'{misc_data_path}nino-3/nino-3_1980-2019.nc'

    if os.path.exists(out_fp):
        if not overwrite_flag:
            return  # don't continue if file already exists and overwrite_flag is False

    # load csv file
    df = pd.read_csv(in_fp)

    # rename first column to year
    df.rename(columns={'Unnamed: 0': 'year'}, inplace=True)

    # melt dataframe for single time dimension
    df_melted = df.melt(id_vars='year', var_name='month', value_name='nino-3')

    # create datetime index
    df_melted['time'] = pd.to_datetime(df_melted['year'].astype(str) + df_melted['month'], format='%Y%B')

    # drop redundant year and month column
    df_melted = df_melted.drop(columns=['year', 'month'])

    # set datetime index as df
    df_melted = df_melted.set_index('time')

    # convert to xarray dataset
    ds = df_melted.to_xarray()

    # ensure dtype is float64
    ds['nino-3'] = ds['nino-3'].astype('float64')

    # save to netcdf
    ds.to_netcdf(out_fp)

In [16]:
# run the nino-3 code
if __name__ == '__main__':
    nino_3_extractor()